# Content



 * 1 Study file
 * 2 Splitting data into samples
 * 3 Model study
 * Intermediate conclusions
 * 4 Checking the best model on the test set
 * 5 Checking the best model for adequacy
 * Conclusion


<div style="border:2px solid Black; padding:20px;">
<h1> Fare Recommendations</h1>
We have at our disposal data on the behavior of customers who have already switched to these tariffs. You need to build a model for the classification problem that will select the appropriate rate. Data preprocessing is not required - it is already done.

Let's build a model with the largest *accuracy* value. 
</div>


## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('datasets/users_behavior.csv')

print(df.info());

print(df.head());

print(df.describe());

print(df.value_counts());

print(df.duplicated());

df.hist(figsize=(15,10));

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/users_behavior.csv'


## Divide data into samples

We split the data into 3 parts in proportions 3:1:1

Training set - 60%, Validation set - 20%, Test set - 20%

In [ ]:
df_train, df_test_plus_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_test_plus_valid, test_size=0.5, random_state=12345)
print(df_train)
print(df_test)
print(df_valid)

      calls  minutes  messages   mb_used  is_ultra
3027   60.0   431.56      26.0  14751.26         0
434    33.0   265.17      59.0  17398.02         0
1226   52.0   341.83      68.0  15462.38         0
1054   42.0   226.18      21.0  13243.48         0
1842   30.0   198.42       0.0   8189.53         0
...     ...      ...       ...       ...       ...
2817   12.0    86.62      22.0  36628.85         1
546    65.0   458.46       0.0  15214.25         1
382   144.0   906.18       0.0  25002.44         1
2177   38.0   301.27      37.0  28914.24         1
482   160.0  1084.05      95.0  36843.92         1

[1928 rows x 5 columns]
      calls  minutes  messages   mb_used  is_ultra
1386   92.0   536.96      18.0  20193.90         0
3124   40.0   286.57      17.0  17918.75         0
1956   81.0   531.22      56.0  17755.06         0
2286   67.0   460.76      27.0  16626.26         0
3077   22.0   120.09      16.0   9039.57         0
...     ...      ...       ...       ...       ...
1999  

In [ ]:
train, valid, test = np.split(df.sample(frac=1, random_state=12345),
                              [int(.6*len(df)), int(.8*len(df))])
print(train)
print(valid)
print(test)

      calls  minutes  messages   mb_used  is_ultra
1415   82.0   507.89      88.0  17543.37         1
916    50.0   375.91      35.0  12388.40         0
1670   83.0   540.49      41.0   9127.74         0
686    79.0   562.99      19.0  25508.19         1
2951   78.0   531.29      20.0   9217.25         0
...     ...      ...       ...       ...       ...
2104  118.0   844.52      64.0  19459.08         1
2442   93.0   639.07      29.0  26770.62         1
1265   66.0   437.75      22.0  25108.55         0
1635   68.0   464.62       6.0   5810.80         0
848    61.0   398.63      76.0  12230.87         0

[1928 rows x 5 columns]
      calls  minutes  messages   mb_used  is_ultra
3116   83.0   509.32      69.0  19355.79         0
3018   49.0   302.43      38.0  20974.22         0
2443   91.0   549.56      41.0  26788.18         1
757    81.0   531.28     109.0  19496.93         1
752    79.0   577.11      46.0  30981.68         0
...     ...      ...       ...       ...       ...
824   

## Exploring models 

In [ ]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Decision tree model

# <make a loop for max_depth from 1 to 20>
for depth in range(1,21):
    model =DecisionTreeClassifier(random_state=12345,max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))


max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255
max_depth = 6 : 0.776049766718507
max_depth = 7 : 0.7993779160186625
max_depth = 8 : 0.7931570762052877
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7884914463452566
max_depth = 11 : 0.7744945567651633
max_depth = 12 : 0.7807153965785381
max_depth = 13 : 0.7713841368584758
max_depth = 14 : 0.76049766718507
max_depth = 15 : 0.7713841368584758
max_depth = 16 : 0.749611197511664
max_depth = 17 : 0.7480559875583204
max_depth = 18 : 0.749611197511664
max_depth = 19 : 0.7433903576982893
max_depth = 20 : 0.7325038880248833


- В данном примере видно как  модель "дерево решений2 переобучилась, после глубины дерева - 7 точность начнает падать.

In [ ]:
# Random forest model

#Calculate the accuracy for a model consisting of 5 trees:
best_model = None
best_result = 0
for est in range(1, 6):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) # train the model with the given number of trees
    model.fit(features_train,target_train) # train the model on the training set
    result = model.score(features_valid, target_valid) # calculate the quality of the model on the validation set
    if result > best_result:
        best_model = model
        best_result = result # store the best value of the accuracy metric on the validation data
# print('number of trees:', est, 'Accuracy:', result)

print("Accuracy of the best model on the validation set:", best_result)


#Calculate the accuracy for a model consisting of 10 trees:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) # train the model with the given number of trees
    model.fit(features_train,target_train) # train the model on the training set
    result = model.score(features_valid, target_valid) # calculate the quality of the model on the validation set
    if result > best_result:
        best_model = model
        best_result = result # store the best value of the accuracy metric on the validation data
    print('number of trees:', est, 'Accuracy:', result)

print("Accuracy of the best model on the validation set:", best_result)

#Calculate the accuracy for a model consisting of 100 trees:
best_model = None
best_result = 0
for est in range(1, 101):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) # train the model with the given number of trees
    model.fit(features_train,target_train) # train the model on the training set
    result = model.score(features_valid, target_valid) # calculate the quality of the model on the validation set
    if result > best_result:
        best_model = model
        best_result = result # store the best value of the accuracy metric on the validation data
    print('number of trees :', est, 'Accuracy:', result)

print("Accuracy of the best model on the validation set:", best_result)


Accuracy наилучшей модели на валидационной выборке: 0.7729393468118196
количество деревьев: 1 Accuracy: 0.7402799377916018
количество деревьев: 2 Accuracy: 0.7589424572317263
количество деревьев: 3 Accuracy: 0.7573872472783826
количество деревьев: 4 Accuracy: 0.7729393468118196
количество деревьев: 5 Accuracy: 0.7667185069984448
количество деревьев: 6 Accuracy: 0.7791601866251944
количество деревьев: 7 Accuracy: 0.7807153965785381
количество деревьев: 8 Accuracy: 0.7869362363919129
количество деревьев: 9 Accuracy: 0.7838258164852255
количество деревьев: 10 Accuracy: 0.7807153965785381
Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129
количество деревьев: 1 Accuracy: 0.7402799377916018
количество деревьев: 2 Accuracy: 0.7589424572317263
количество деревьев: 3 Accuracy: 0.7573872472783826
количество деревьев: 4 Accuracy: 0.7729393468118196
количество деревьев: 5 Accuracy: 0.7667185069984448
количество деревьев: 6 Accuracy: 0.7791601866251944
количество деревьев: 7 Ac

- In this example, you can see how this model does not retrain with an increase in trees, the accuracy increases with the number of trees, but the model becomes heavier.

In [ ]:
# Logistic regression model
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
model.fit(features_train,target_train) # train the model on the training set
result = model.score(features_valid, target_valid) # calculate the quality of the model on the validation
result

0.6842923794712286

### Brief interim conclusions:
At this stage, the following models were trained:
- decision tree,
- random forest,
- logistic regression.


Each has its own merits and demerits. After evaluating the models, we can draw the following conclusions on the quality (accuracy) and speed of each:

1. Quality (accuracy). This is the most important criterion for business: the higher the quality, the more profit the product brings.
Random forest has the highest quality: instead of one decision tree, the whole ensemble is used.
In second place is the decision tree. If the depth is less than seven, it is undertrained, when it is more, it is retrained. Logistic regression has the lowest prediction quality. The model is simple, which means that retraining does not threaten it.


2. Speed ​​of work. An equally significant criterion: if the service is slow, the outflow of users cannot be avoided.
Logistic regression has a high speed of work: it has the fewest parameters.
The speed of the decision tree is also high and depends on the depth. Remember, in experiments, the best quality of the model was obtained at a depth equal to 7. The model received the answer to the question in just seven checks of feature values ​​- and this is very fast!
The random forest is the slowest of all: the more trees, the slower the model works.

## Check the model on the test set

In [ ]:
#Calculate the accuracy for a model consisting of 100 trees on a test set:
best_model = None
best_result = 0
for est in range(1, 101):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) # train the model with the given number of trees
    model.fit(features_train,target_train) # train the model on the training set
    result = model.score(features_test, target_test) # calculate the quality of the model on the validation set
    if result > best_result:
        best_model = model
        best_result = result # store the best value of the accuracy metric on the validation data
    print('number of trees: ', est, 'Accuracy: ', result)

print("Accuracy of the best model on the test set: ", best_result)

количество деревьев: 1 Accuracy: 0.7107309486780715
количество деревьев: 2 Accuracy: 0.7636080870917574
количество деревьев: 3 Accuracy: 0.7387247278382582
количество деревьев: 4 Accuracy: 0.7713841368584758
количество деревьев: 5 Accuracy: 0.749611197511664
количество деревьев: 6 Accuracy: 0.7807153965785381
количество деревьев: 7 Accuracy: 0.7682737169517885
количество деревьев: 8 Accuracy: 0.7822706065318819
количество деревьев: 9 Accuracy: 0.7729393468118196
количество деревьев: 10 Accuracy: 0.7853810264385692
количество деревьев: 11 Accuracy: 0.7838258164852255
количество деревьев: 12 Accuracy: 0.7869362363919129
количество деревьев: 13 Accuracy: 0.7822706065318819
количество деревьев: 14 Accuracy: 0.7838258164852255
количество деревьев: 15 Accuracy: 0.7838258164852255
количество деревьев: 16 Accuracy: 0.7869362363919129
количество деревьев: 17 Accuracy: 0.7869362363919129
количество деревьев: 18 Accuracy: 0.7931570762052877
количество деревьев: 19 Accuracy: 0.7884914463452566
кол


- "Accuracy" of the best model on the test set is slightly lower than "Accuracy" of the best model on the validation set, but at the same time quite high above 0.75.

## Check the models for adequacy

In [ ]:
relation= ((df['is_ultra'].loc[df['is_ultra'] == 0].count())/
(df['is_ultra'].count()))

print('%.5f'%relation)


0.69353


In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)

dummy_clf.fit(df['is_ultra'], df['is_ultra'])

dummy_clf.score(df['is_ultra'], df['is_ultra'])




0.693528313627878

- We can conclude that the model is adequate because the most common class "occupies" 69% (2229 / 3214 == 0.693528313627878). This is the control data for a comparative assessment of the "best" model we have built. Our best model exceeds this score.

<div style="border:2px solid Black; padding:20px;">
<h1> Conclusion </h1>
- We had at our disposal data on the behavior of customers who had already switched to these tariffs (from the draft course "Statistical Data Analysis").

- In the project, models were built for the classification task, which will select the appropriate tariff. The preprocessing has already been done.

- A model was built with the largest *accuracy* value, the proportion of correct answers exceeds 0.75.

- In this example, you can see how the "decision tree2" model was retrained, after the depth of the tree - 7, the accuracy starts to fall.

- Accuracy of the best "Random Forest" model consisting of 100 trees on the validation set: 0.7962674961119751

- "Accuracy" of the best model consisting of 100 trees on the test set: 0.7947122861586314 (slightly lower than the "Accuracy" of the best model on the validation set, but at the same time quite high above 0.75).

- We can conclude that the model is adequate because the most common class "occupies" 69% (2229 / 3214 == 0.693528313627878). This is the control data for a comparative assessment of the "best" model we have built. Our best model exceeds this speed.
</div>